# Library

In [27]:
import pickle

import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load data into a pandas DataFrame
# Pastiin pathnya sesuai sama file nya dimana ya
a = pd.read_csv('/content/drive/MyDrive/Capstone Project/dataset-ayam.csv')
b = pd.read_csv('/content/drive/MyDrive/Capstone Project/dataset-ikan.csv')
c = pd.read_csv('/content/drive/MyDrive/Capstone Project/dataset-sapi.csv')
d = pd.read_csv('/content/drive/MyDrive/Capstone Project/dataset-tahu.csv')
e = pd.read_csv('/content/drive/MyDrive/Capstone Project/dataset-telur.csv')
f = pd.read_csv('/content/drive/MyDrive/Capstone Project/dataset-tempe.csv')
g = pd.read_csv('/content/drive/MyDrive/Capstone Project/dataset-udang.csv')

In [4]:
ayam = pd.DataFrame(a)
ikan = pd.DataFrame(b)
sapi = pd.DataFrame(c)
tahu = pd.DataFrame(d)
telur = pd.DataFrame(e)
tempe = pd.DataFrame(f)
udang = pd.DataFrame(g)

In [5]:
indofood = pd.concat([ayam, ikan, sapi, tahu, telur, tempe, udang])
indofood = pd.DataFrame(indofood)

# Check for Null Data and Clean It


In [6]:
indofood.isnull().sum().sum()

68

In [7]:
print(indofood.isnull().sum())

Title           0
Ingredients    35
Steps          33
Loves           0
URL             0
dtype: int64


In [8]:
df_clean = indofood.dropna()
print(len(df_clean))

13710


In [9]:
indofood_new = pd.DataFrame(df_clean)

In [10]:
indofood_new.isnull().sum().sum()

0

# Clustering with K-Means


In [14]:
indofood_new['text'] = indofood_new[['Title', 'Ingredients', 'Steps']].apply(lambda x: ' '.join(x), axis=1)

In [15]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(indofood_new['text'])

In [16]:
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=3, random_state=0)

In [37]:
mse = kmeans.inertia_
print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 11865.752781739964


In [17]:
labels = kmeans.labels_

In [18]:
indofood_new['category'] = labels

# Membagi dataframe berdasarkan kategori
kategori_murah = indofood_new[indofood_new['category'] == 0]
kategori_terjangkau = indofood_new[indofood_new['category'] == 1]
kategori_mahal = indofood_new[indofood_new['category'] == 2]

In [19]:
indofood_new.head()

,Title,Ingredients,Steps,Loves,URL,text,category
0,Ayam Woku Manado,1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1,/id/resep/4473027-ayam-woku-manado,Ayam Woku Manado 1 Ekor Ayam Kampung (potong 1...,1
1,Ayam goreng tulang lunak,1 kg ayam (dipotong sesuai selera jangan kecil...,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1,/id/resep/4471956-ayam-goreng-tulang-lunak,Ayam goreng tulang lunak 1 kg ayam (dipotong s...,1
2,Ayam cabai kawin,1/4 kg ayam--3 buah cabai hijau besar--7 buah ...,Panaskan minyak di dalam wajan. Setelah minyak...,2,/id/resep/4473057-ayam-cabai-kawin,Ayam cabai kawin 1/4 kg ayam--3 buah cabai hij...,1
3,Ayam Geprek,250 gr daging ayam (saya pakai fillet)--Secuku...,Goreng ayam seperti ayam krispi--Ulek semua ba...,10,/id/resep/4473023-ayam-geprek,Ayam Geprek 250 gr daging ayam (saya pakai fil...,1
4,Minyak Ayam,400 gr kulit ayam & lemaknya--8 siung bawang p...,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4,/id/resep/4427438-minyak-ayam,Minyak Ayam 400 gr kulit ayam & lemaknya--8 si...,1


# Decode Category Column


In [20]:
decode_map = {0: "Murah", 1: "Terjangkau", 2: "Mahal"}
def decode_category(label):
    return decode_map[int(label)]

indofood_new.category = indofood_new.category.apply(lambda x: decode_category(x))

In [21]:
indofood_new.head()

,Title,Ingredients,Steps,Loves,URL,text,category
0,Ayam Woku Manado,1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...,Cuci bersih ayam dan tiriskan. Lalu peras jeru...,1,/id/resep/4473027-ayam-woku-manado,Ayam Woku Manado 1 Ekor Ayam Kampung (potong 1...,Terjangkau
1,Ayam goreng tulang lunak,1 kg ayam (dipotong sesuai selera jangan kecil...,"Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...",1,/id/resep/4471956-ayam-goreng-tulang-lunak,Ayam goreng tulang lunak 1 kg ayam (dipotong s...,Terjangkau
2,Ayam cabai kawin,1/4 kg ayam--3 buah cabai hijau besar--7 buah ...,Panaskan minyak di dalam wajan. Setelah minyak...,2,/id/resep/4473057-ayam-cabai-kawin,Ayam cabai kawin 1/4 kg ayam--3 buah cabai hij...,Terjangkau
3,Ayam Geprek,250 gr daging ayam (saya pakai fillet)--Secuku...,Goreng ayam seperti ayam krispi--Ulek semua ba...,10,/id/resep/4473023-ayam-geprek,Ayam Geprek 250 gr daging ayam (saya pakai fil...,Terjangkau
4,Minyak Ayam,400 gr kulit ayam & lemaknya--8 siung bawang p...,Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...,4,/id/resep/4427438-minyak-ayam,Minyak Ayam 400 gr kulit ayam & lemaknya--8 si...,Terjangkau


# Input Data 1

In [24]:
def get_recommendations_by_price_and_ingredients(price_input, ingredients, top_k=5):
    if price_input <= 30000:
        price_category = 'Murah'
    elif 30000 < price_input <= 90000:
        price_category = 'Terjangkau'
    else:
        price_category = 'Mahal'

    # Filter dataset berdasarkan kategori harga
    filtered_df = indofood_new[indofood_new['category'] == price_category]

    filtered_df['text'] = filtered_df['Title'] + ' ' + filtered_df['Ingredients'] + ' ' + filtered_df['Steps']

    vectorizer = TfidfVectorizer(analyzer='word',
                                 ngram_range=(1, 2),
                                 min_df=0.003,
                                 max_df=0.5,
                                 max_features=5000)

    tfidf_matrix = vectorizer.fit_transform(filtered_df['text'])
    tfidf_feature_names = vectorizer.get_feature_names_out()

    input_vector = vectorizer.transform([ingredients])

    similarity_scores = cosine_similarity(input_vector, tfidf_matrix)

    top_indices = similarity_scores.argsort()[0][-top_k:][::-1]

    recommendations = filtered_df.iloc[top_indices][['Title', 'Ingredients', 'Steps']]

    return recommendations

In [25]:
input_price = int(input("Masukkan nominal: "))
input_ingredients = input("Masukkan bahan yang diinginkan: ")

Masukkan nominal: 53000
Masukkan bahan yang diinginkan: tepung terigu, garama


In [28]:
recommendations = get_recommendations_by_price_and_ingredients(input_price, input_ingredients)

print("Rekomendasi berdasarkan harga dan bahan:")
print(recommendations)

<ipython-input-24-9ffa55bc15cc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['text'] = filtered_df['Title'] + ' ' + filtered_df['Ingredients'] + ' ' + filtered_df['Steps']


Rekomendasi berdasarkan harga dan bahan:
                                      Title  \
891                          Chicken nugget   
1892  Crispy fried egg (telor dadar crispy)   
305                             Tahu krispi   
423                 Nugget Ayam Keju Wortel   
359               Ayam goreng tepung tasty🤤   

                                            Ingredients  \
891   300 gram ayam fillet--1 buah wortel--50 ml air...   
1892  Bahan Adonan Telur Dadar :--2 butir telur--7 s...   
305   Tahu--Tepung kobe original--Tepung terigu--Min...   
423   250 gr ayam cincang--75 gr keju cheddar--1 bh ...   
359   300 gr ayam kurang lebih dipotong2--Bumbu ungk...   

                                                  Steps  
891   Blender daging ayam yang sudah di potong poton...  
1892  Adonan telur : Masukkan tepung terigu dalam ma...  
305   Masukkan 5 sendok makan tepung kobe dan 2 send...  
423   Siapkan daging ayam cincang pada wadah, masuka...  
359   Haluskan bumbu halus camp

# Input Data 2



In [32]:
cat = input("Masukkan bahan yang diinginkan: ")

Masukkan bahan yang diinginkan: tepung terigu


In [33]:
tmp = indofood_new[indofood_new['Ingredients'].str.contains(cat)].head(10)
tmp[['Title', 'Ingredients', 'Steps', 'category']]

,Title,Ingredients,Steps,category
9,Ayam Saos Asam Manis Simple,1/4 kg Ayam bagian dada fillet (Potong dadu)--...,Lumuri ayam yg sdh dipotong dadu dgn garam & p...,Terjangkau
10,Mie Ayam Homemade by suami😂😂😂,Bahan mie ayam :--1 kg tepung terigu proteiin ...,"Kocok telur masukan garam n minyak goreng,aduk...",Terjangkau
20,Ayam Koloke,"250 gr ayam fillet, potong-potong--2 sdm tepun...",Marinade ayam dgn tepung bumbu serbaguna yg su...,Terjangkau
27,Nugget Ayam Wortel,250 gram daging ayam giling--1 buah wortel uku...,Potong-potong bawang bombay dan bawang putih. ...,Terjangkau
33,Ayam goreng krutuk,tepung terigu--Air matang--ayam (yg sudah dire...,Campur tepung terigu(kurang lebih 1/2 mangkuk ...,Terjangkau
37,Mie Geprek (Mie + ayam geprek),1 bungkus mie burung dara--1 ons dada ayam--te...,"Rebus mie burung dara, tiriskan--Geprek bawang...",Terjangkau
43,Ayam goreng tepung simple,500 gram ayam dada potong2--6 siung bawang put...,"Potong ayam sesuai selera, aku pake ayam dada ...",Terjangkau
48,Ayam Tepung KFC super gurih,4 potong ayam (paha atas atau paha bawah)--1 S...,"Marinade Ayam dengan air lemon, black papper D...",Terjangkau
51,Risoles rogut ayam,"Bahan isian:--3 buah kentang, kukus dan tumbuk...","Aduk rata kentang kukus yang sudah dihaluskan,...",Terjangkau
53,Ayam tepung,1 kg ayam--500 gram tepung terigu--secukupnya ...,"Bersih kan ayam potong jadi 14--Haluskan,bawan...",Terjangkau


In [34]:
# Menginputkan nominal dari pengguna
nominal_input = int(input("Masukkan nominal: "))

Masukkan nominal: 25000


In [35]:
# Mengkategorikan berdasarkan nominal
if nominal_input <= 30000:
    category = 'Murah'
elif 30000 < nominal_input <= 90000:
    category = 'Terjangkau'
else:
    category = 'Mahal'

In [36]:
# Filter dan menampilkan data berdasarkan kategori
df_filtered = indofood_new.loc[indofood_new['category'] == category, ['Title', 'Ingredients', 'Steps']]

if not df_filtered.empty:
    print(f"Data yang termasuk kategori '{category}':")
    print(df_filtered[['Title', 'Ingredients', 'Steps']])
else:
    print(f"Tidak ada data yang termasuk kategori '{category}'")

Data yang termasuk kategori 'Murah':
                                            Title  \
101               Ayam Dan Tempe Goreng Serundeng   
171                      Ayam goreng sambal matah   
403               Sayur tempe, terong & hati ayam   
530                 Ayam & Tempe Geprek Keju Melt   
540                  Ayam dan Tempe Goreng Kunyit   
...                                           ...   
1646                            Oseng Tempe Udang   
1758  Tauco Udang 🦐 (Terong,Tempe,Kacang Panjang)   
1799            Udang & Tempe bakar sambel kencur   
1811                     Tumis kacang tempe udang   
1912         Udang dan tempe penyet sambel terasi   

                                            Ingredients  \
101   1 dada ayam--1/2 kelapa parut--2 tempe (potong...   
171   Ayam kampung paha dan dada--1 sachet bumbu aya...   
403   5 hati ayam (rebus)--1 papan tempe (potong2)--...   
530   4 Paha atas Ayam--6 iris tempe--Bahan bumbu ay...   
540   1/2 ekor ayam, potong 6--

# Save Model

In [ ]:
# save the model to disk
filename = 'finmodel.sav'
pickle.dump(kmeans, open(filename, 'wb'))

In [38]:
indofood_new.to_csv('dataresep_new.csv', index=False)